In this notebook, I solve the alternative HW1 for Kyle Herkenhoff's . In this homework, we are asked to solve a simple Bewley model.

In [2]:
using LinearAlgebra, Optim, Makie, Roots, Interpolations

In [31]:
# Params = [beta, gamma]
Params = Vector{Float64}([0.9, 1.0])
# Sizes = [grid_expansion_factor, number_of_states, number_of_actions]
Sizes = Vector{Int64}([2, 2, 101])

3-element Vector{Int64}:
   2
   2
 101

In [4]:
function expanding_grid(S)

    zero_grid = LinRange(0, 1, S[3])

    asset_grid = zeros(S[3], 1)

    a_low = 0.0

    a_high = 200.0

    for i in 1:S[3]

        asset_grid[i] = (zero_grid[i]^S[1]) * (a_high - a_low) + a_low

    end

    return asset_grid

end

expanding_grid (generic function with 1 method)

In [5]:
function utility(c, P)

    if P[2] == 1

        return log(c)

    else

        return c^(1-P[2])/(1-P[2])

    end

end

utility (generic function with 1 method)

In [6]:
function states(S)

    states = range(0.95, 1.05, S[2])

    transition = zeros(S[2], S[2])

    for i in 1:S[2]

        for j in 1:S[2]

            transition[i,j] = 1/S[2]

        end
    
    end

    return states, transition

end

states (generic function with 1 method)

In [7]:
function generate_matrices(S, P)

    value = zeros(S[3], S[2])

    policy = zeros(S[3], S[2])

    distance = zeros(S[3], S[2])

    return value, policy, distance

end

generate_matrices (generic function with 1 method)

In [109]:
function Bellman_grid_search(S, P, tolerance, max_iterations)

    iter = 0

    println("Iterating...")

    state, transition = states(S)

    bond_grid = expanding_grid(S)

    error = tolerance + 1

    Val, Pol, Distance = generate_matrices(S, P)

    V_old = similar(Val)

    Pol_old = similar(Pol)

    expected_value = similar(Val)

    #while (error > tolerance) && (max_iterations > 0)

    for i in 1:S[3]

        consumption = bond_grid[i] .+ state .- bond_grid'

        c_positive = consumption .> 0

        consumption_positive = consumption .* c_positive

        #expected_value = V_old * transition

        println(expected_value)

        #Val[i,:] = utility.(consumption_positive', P) .+ (P[1] * expected_value')

    end
    
    return Val, Pol

end

Bellman_grid_search (generic function with 1 method)

In [108]:
V, P = Bellman_grid_search(Sizes, Params, 0.01, 1000)

([0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 0.0])